In [ ]:
from datetime import datetime
from dateutil import relativedelta

now = datetime(year=2023, month=12, day=18, hour=12, minute=18, second=0)

print('현재시간:' + str(now))
print('------------월 연산-------------')
print(now + relativedelta.relativedelta(month=1))
print(now.replace(month=1))

In [ ]:
import snowflake.connector
import pandas as pd

# 스노우플레이크 연결 설정
conn = snowflake.connector.connect(
    user='hdhsdwload',                # 스노우플레이크 사용자명
    password='Ghatyvld1!',            # 비밀번호
    account='dz48560.ap-northeast-2.aws',              # 계정 (예: xy12345.ap-southeast-2)
    warehouse='DW_LOAD_WH'          # 스키마 이름
)

try:
    # 커서 생성
    cursor = conn.cursor()

    # SQL 쿼리 실행
    query = "SELECT COUNT(*) FROM DW_LOAD_DB.HDHS_CU.CU_ITNT_CUST_GRD_1_INF"  # 조회할 테이블과 조건
    query2 = "alter stage DW_LOAD_DB.CONFIG.STG_AWS_S3 refresh"  # 조회할 테이블과 조건
    query3 = f"""
        SELECT 
           *
        FROM 
            DW_LOAD_DB.CONFIG.V_S3_LIST A 
        """
    cursor.execute(query)
    print (cursor.fetchall())
    
    cursor.execute(query2)
    print (type(cursor.fetchone()[1]))
    
    colums = [row[0] for row in cursor.fetchall()]
    df = pd.read_sql(query3, conn)

    # 결과 출력

finally:
    # 연결 닫기
    cursor.close()
    conn.close()
    
df

In [ ]:
df =pd.DataFrame(df)

In [ ]:
df

In [ ]:
df.to_parquet('df.parquet',engine='pyarrow', index=False)

In [ ]:
df['TABLE_NAME'][0]

In [ ]:
# 스노우플레이크 연결 설정
conn = snowflake.connector.connect(
    user='hdhsdwload',                # 스노우플레이크 사용자명
    password='Ghatyvld1!',            # 비밀번호
    account='dz48560.ap-northeast-2.aws',              # 계정 (예: xy12345.ap-southeast-2)
    warehouse='DW_LOAD_WH'          # 스키마 이름
)

cursor = conn.cursor()

# for i,row in df.iterrows():
#     if df['TABLE_SCHEMA'][i] == "HDHS_OD" and df['TABLE_NAME'][i] != "OD_BASKT_INF":
#         
#         query5 = f"""
#         CALL DW_LOAD_DB.CONFIG.PROC_CDC_SUB('DW_LOAD_DB','HDHS_OD','{df['TABLE_NAME'][i]}','{df['FILE_NAME']}','20241223120000','20241223150000')
#         """
#         print(query5)
#         cursor.execute(query5)
#         print(cursor.fetchall())
        
query5 = f""" 
    CALL DW_LOAD_DB.CONFIG.PROC_CDC_SUB(
        'DW_LOAD_DB',
        'HDHS_OD',
        '{df['TABLE_NAME'][1]}',""" + str("''2024/12/23/CU_ARS_LDIN_MST_CRYPT_20241223_030000.csv','2024/12/23/20241223_040000.csv''") +""",
        '20241223120000',
        '20241223150000'
    )
    """
print(query5)
cursor.execute(query5)
print(cursor.fetchall())
        
        
cursor.close()
conn.close() 

In [ ]:
fmrom airflow.providers.snowflake.operators.snowflake import SnowflakeOperator

CALL_PROCEDURE_QUERY = """
CALL DW_LOAD_DB.CONFIG.PROC_CDC_SUB(
    'DW_LOAD_DB',
    'HDHS_OD',
    '{{ params.table_name }}',
    '{{ params.file_name }}',
    '{{ params.start_time }}',
    '{{ params.end_time }}'
)
"""

call_procedure = SnowflakeOperator(
    task_id='call_snowflake_procedure',
    snowflake_conn_id='my_snowflake_conn',
    sql=CALL_PROCEDURE_QUERY,
    params={
        'table_name': {df['TABLE_NAME'][1]},
        'file_name': {df['FILE_NAME'][1]},
        'start_time': '20241223120000',
        'end_time': '20241223150000'
    },
)

print(call_procedure)


In [6]:
import pandas as pd

src_columns =['PROC_TIME','TRED','CALL_ID','MESG','Q_WAIT_TIME','STON','VDN_DIAL_NO','UCID','IN_CALL_NO']
tgt_columns =['UCID','ACD','PROC_DATE','PROC_TIME','TRED','CALL_ID','MESG','Q_WAIT_TIME','STON','VDN_DIAL_DVCD','VDN_DIAL_NO','IN_CALL_NO','ETL_DTM']
file_path="hmall_1225.dmp"
# 파일을 Pandas DataFrame으로 변환
try:
    # 쉼표로 구분된 데이터를 읽기 (UTF-8 또는 EUC-KR 등의 인코딩을 시도)
    df = pd.read_csv(file_path, encoding='utf-8', header=None)
except UnicodeDecodeError:
    # 만약 UTF-8로 읽히지 않으면 EUC-KR로 재시도
    df = pd.read_csv(file_path, encoding='euc-kr', header=None)

df = df.iloc[:, :-3]

df.columns = src_columns

df



,PROC_TIME,TRED,CALL_ID,MESG,Q_WAIT_TIME,STON,VDN_DIAL_NO,UCID,IN_CALL_NO
0,00:00:00.146,2140,c6100,상담연결(AgentVDN->Agent),BSR,i01093101121,v60324,U10001044731703429962,A01093101121
1,00:00:01.146,2328,c6097,상담연결(AgentVDN->Agent),,i01084865650,v60717,U10002060531703429968,A01084865650
2,00:00:02.114,22a8,c6090,상담연결(AgentVDN->Agent),,i01029385307,v60105,U10002060901703429993,A01029385307
3,00:00:02.614,17e4,c6010,호종료,,s3167,v,U,A
4,00:00:02.614,205c,c5801,수신(Inbound),,s3167,v3416,U10002058011703429734,A01073785912
...,...,...,...,...,...,...,...,...,...
118551,23:45:58.331,0dc8,c2656,상담연결(AgentVDN->Agent),,i10512,v62044,U10002026541735137940,A0618633452
118552,23:58:01.657,2fc0,c,IVR상담연결(IVR->CTI),,i10085,v64601,U,A01021140996
118553,23:58:44.695,30ec,c,IVR상담연결(IVR->CTI),,i10184,v62036,U,A01075774322
118554,23:58:46.398,3370,c2900,상담연결(AgentVDN->Agent),,i10184,v62036,U10002028831735138697,A01075774322


In [7]:
from datetime import timedelta

def create_target_dataframe(df, tgt_columns, execution_date):
    # 실행 날짜 계산
    proc_date = (execution_date - timedelta(days=1)).strftime('%Y%m%d')
    proc_date_yyyymmmm = proc_date[:8]  # YYYYMM 포맷으로 가져오기
    
    # 데이터 변환
    transformed_data = {
        'UCID': df['UCID'].str[1:21],  # SUBSTR(UCID,2,20)
        'ACD': 2,                      # 고정값
        'PROC_DATE': proc_date_yyyymmmm, # execution_date - 1일에서 YYYYMMMM 추출
        'PROC_TIME': df['PROC_TIME'].str.lstrip(),  # LTRIM(PROC_TIME)
        'TRED': df['TRED'],            # 그대로 유지
        'CALL_ID': df['CALL_ID'].str[1:6],  # SUBSTR(CALL_ID,2,5)
        'MESG': df['MESG'],            # 그대로 유지
        'Q_WAIT_TIME': df['Q_WAIT_TIME'].apply(
            lambda x: x[1:] if x.startswith('Q') else x  # IIF 조건 적용
        ),
        'STON': df['STON'].str[1:5],   # SUBSTR(STON,2,4)
        'VDN_DIAL_DVCD': df['VDN_DIAL_NO'].str[0].str.upper(),  # UPPER(SUBSTR(VDN_DIAL_NO,1,1))
        'VDN_DIAL_NO': df['VDN_DIAL_NO'].str[1:],  # SUBSTR(VDN_DIAL_NO,2,LENGTH(VDN_DIAL_NO))
        'IN_CALL_NO': df['IN_CALL_NO'].str[1:],  # SUBSTR(IN_CALL_NO,2,LENGTH(IN_CALL_NO))
        'ETL_DTM': execution_date.strftime('%Y-%m-%d %H:%M:%S')  # execution_date
    }
    target_df = pd.DataFrame(transformed_data, columns=tgt_columns)
    return target_df

execution_date = pd.Timestamp('2023-12-26')

# 변환된 데이터프레임 생성
result_df = create_target_dataframe(df, tgt_columns, execution_date)

# 결과 확인
result_df

,UCID,ACD,PROC_DATE,PROC_TIME,TRED,CALL_ID,MESG,Q_WAIT_TIME,STON,VDN_DIAL_DVCD,VDN_DIAL_NO,IN_CALL_NO,ETL_DTM
0,10001044731703429962,2,20231225,00:00:00.146,2140,6100,상담연결(AgentVDN->Agent),BSR,0109,V,60324,01093101121,2023-12-26 00:00:00
1,10002060531703429968,2,20231225,00:00:01.146,2328,6097,상담연결(AgentVDN->Agent),,0108,V,60717,01084865650,2023-12-26 00:00:00
2,10002060901703429993,2,20231225,00:00:02.114,22a8,6090,상담연결(AgentVDN->Agent),,0102,V,60105,01029385307,2023-12-26 00:00:00
3,,2,20231225,00:00:02.614,17e4,6010,호종료,,3167,V,,,2023-12-26 00:00:00
4,10002058011703429734,2,20231225,00:00:02.614,205c,5801,수신(Inbound),,3167,V,3416,01073785912,2023-12-26 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
118551,10002026541735137940,2,20231225,23:45:58.331,0dc8,2656,상담연결(AgentVDN->Agent),,1051,V,62044,0618633452,2023-12-26 00:00:00
118552,,2,20231225,23:58:01.657,2fc0,,IVR상담연결(IVR->CTI),,1008,V,64601,01021140996,2023-12-26 00:00:00
118553,,2,20231225,23:58:44.695,30ec,,IVR상담연결(IVR->CTI),,1018,V,62036,01075774322,2023-12-26 00:00:00
118554,10002028831735138697,2,20231225,23:58:46.398,3370,2900,상담연결(AgentVDN->Agent),,1018,V,62036,01075774322,2023-12-26 00:00:00
